In [20]:
import pandas as pd

In [21]:
# Setting the working directory 
import os
os.chdir(r"C:\Users\olivi\OneDrive\Documents\Personal Projects\Vaccination Coverage Among Adults Dashboard") 

In [22]:
vacc = pd.read_csv("Vaccination_Coverage_Among_Adults_18_Years.csv")
vacc.head()

,Vaccine,Dose,Geography Type,Geography,Survey Year,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Tetanus,Tdap,States/Local Areas,Maryland,2019,>=18 Years,Overall,44.5,42.9 to 46.1,8824.0
1,Zoster (Shingles),NaN,States/Local Areas,Maryland,2017,>=60 Years,Overall,43.3,41.5 to 45.0,6232.0
2,Tetanus,Tdap,States/Local Areas,Maryland,2011,18-64 Years,Overall,14.6,11.9 to 17.2,1795.0
3,Tetanus,Tdap,States/Local Areas,Maryland,2010,18-64 Years,Overall,9.8,8.0 to 11.6,2130.0
4,Tetanus,Td or Tdap,States/Local Areas,Maryland,2013,>=18 Years,"Black, Non-Hispanic",57.8,54.6 to 60.9,2144.0


In [23]:
vacc.shape

(66360, 10)

In [24]:
vacc.dtypes

Vaccine            object
Dose               object
Geography Type     object
Geography          object
Survey Year         int64
Dimension Type     object
Dimension          object
Estimate (%)       object
95% CI (%)         object
Sample Size       float64
dtype: object

In [25]:
vacc = vacc[(vacc['Estimate (%)'] != 'NR') & (vacc['Estimate (%)'] != '*')] #Exclude rows 
#where Estimate column is 'NR' or '*', i.e. data was not collected

vacc.shape #Checking new number of rows

(63763, 10)

In [26]:
# Changing data types of Estimate and Sample Size columnns
vacc['Estimate (%)'] = pd.to_numeric(vacc['Estimate (%)']) 
vacc['Sample Size'] = pd.to_numeric(vacc['Sample Size']) 

# Survey Year is left as integer data type since it's values are in the form of year only

In [27]:
# Table 1: Average estimate by Survey Year and Vaccine as average_est_vacc

vacc_overall = vacc[vacc.Dimension == 'Overall'] # Create subset where Dimension = Overall
average_est_vacc = vacc_overall.groupby(['Vaccine', 'Survey Year'], as_index = False).agg(Average_Estimate = ('Estimate (%)', 'mean'))
average_est_vacc.head()

,Vaccine,Survey Year,Average_Estimate
0,Pneumococcal,2008,44.531757
1,Pneumococcal,2009,45.920270
2,Pneumococcal,2010,47.829730
3,Pneumococcal,2011,49.635811
4,Pneumococcal,2012,48.694595


In [28]:
# Pivoted form of average_est_vacc 
average_est_pt = average_est_vacc.pivot_table(index= ['Vaccine'], columns='Survey Year', values='Average_Estimate')
average_est_pt.head()

Survey Year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Vaccine,,,,,,,,,,,,,,,,
Pneumococcal,44.531757,45.920270,47.829730,49.635811,48.694595,49.831081,50.453378,53.015541,53.28750,55.950704,51.667776,52.554541,50.374918,49.731857,49.891513,51.500000
Tetanus,NaN,44.328571,42.114545,46.654762,50.050000,43.871429,46.477143,47.766667,46.60757,49.204762,54.651429,56.329440,58.928571,56.325000,50.944667,57.642857
Zoster (Shingles),NaN,13.480000,14.663333,16.580000,20.819048,NaN,30.760633,36.495833,34.98000,37.820952,37.083333,39.294444,38.244180,42.358333,47.366667,49.853226


In [29]:
# As observed from the pivot table, values for the Zoster (Shingles) vaccine are NaN for 2008 and 2013.

zoster_survey_years = vacc[vacc['Vaccine'] == 'Zoster (Shingles)']['Survey Year'].unique()
zoster_survey_years.sort()
zoster_survey_years

# Hence we see that it is due to missing data for 2008, 2013. Meaning data was not collected for these years(NR or *)

array([2009, 2010, 2011, 2012, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023])

In [30]:
# Table 2: Average Estimate by Geography and Survey Year as est_by_geog
# We need to map each state(Column: 'States/Local Areas') to their HHS Region(Column: 'HHS Region/National')
# to enable hierarchical drill-down in Tableau (HHS region level -> Expand to view at State level)

# Form 2 tables, states and hhs
states = vacc_overall[(vacc_overall['Geography Type'] == 'States/Local Areas')].copy() #to avoid SettingWithCopy warning
hhs = vacc_overall[(vacc_overall['Geography Type'] == 'HHS Region/National')].copy()

# Extract state abbreviations from any hybrid entries e.g. 'TX-Harris County'
states['State'] = states['Geography'].str.split('-').str[0]

# Convert abbreviations to state names
abbr_to_state = {
    'IL': 'Illinois',
    'TX': 'Texas',
    'NY': 'New York',
    'PA': 'Pennsylvania'
}

states['State'] = states['State'].replace(abbr_to_state) #Create State column 

#Alternative: states['State'] = states['State'].map(abbr_to_state).fillna(states['State'])

# Map state to HHS Region
state_to_hhs = {
    
'Connecticut': 'Region 1',
'Maine': 'Region 1',
'Massachusetts': 'Region 1',
'New Hampshire': 'Region 1',
'Rhode Island': 'Region 1',
'Vermont': 'Region 1',

'New Jersey': 'Region 2',
'New York': 'Region 2',
'Puerto Rico': 'Region 2',
'U.S. Virgin Islands': 'Region 2',

'Delaware': 'Region 3',
'District of Columbia': 'Region 3',
'Maryland': 'Region 3',
'Pennsylvania': 'Region 3',
'Virginia': 'Region 3',
'West Virginia': 'Region 3',

'Alabama': 'Region 4',
'Florida': 'Region 4',
'Georgia': 'Region 4',
'Kentucky': 'Region 4',
'Mississippi': 'Region 4',
'North Carolina': 'Region 4',
'South Carolina': 'Region 4',
'Tennessee': 'Region 4',

'Illinois': 'Region 5',
'Indiana': 'Region 5',
'Michigan': 'Region 5',
'Minnesota': 'Region 5',
'Ohio': 'Region 5',
'Wisconsin': 'Region 5',

'Arkansas': 'Region 6',
'Louisiana': 'Region 6',
'New Mexico': 'Region 6',
'Oklahoma': 'Region 6',
'Texas': 'Region 6',

'Iowa': 'Region 7', 
'Kansas': 'Region 7', 
'Missouri': 'Region 7', 
'Nebraska': 'Region 7', 

'Colorado': 'Region 8',
'Montana': 'Region 8',
'North Dakota': 'Region 8',
'South Dakota': 'Region 8',
'Utah': 'Region 8',
'Wyoming': 'Region 8',

'American Samoa': 'Region 9',
'Arizona': 'Region 9',
'California': 'Region 9',
'Commonwealth of the Northern Mariana Islands': 'Region 9',
'Federated States of Micronesia': 'Region 9',
'Guam': 'Region 9',
'Hawaii': 'Region 9',
'Nevada': 'Region 9',
'Republic of Palau': 'Region 9',
'Republic of the Marshall Islands': 'Region 9',

'Alaska': 'Region 10',
'Idaho': 'Region 10',
'Oregon': 'Region 10',
'Washington': 'Region 10'

}  

states['HHS Region'] = states['State'].map(state_to_hhs) #Create HHS Region column

# In the hhs table, create 'State' and 'HHS Region' columns 
hhs['State'] = None 
hhs['HHS Region'] = hhs['Geography'] 

# Combine
est_by_geog = pd.concat([states, hhs], ignore_index=True)


In [31]:
# For reference: 
vacc['Dimension Type'].unique() 
vacc[vacc['Geography Type'] == 'States/Local Areas']['Geography'].unique() #all the unique state names in vacc dataset

array(['Maryland', 'Alabama', 'Pennsylvania', 'Georgia', 'Alaska',
       'Delaware', 'North Carolina', 'North Dakota', 'Arizona',
       'Mississippi', 'Missouri', 'Arkansas', 'Tennessee', 'California',
       'Nevada', 'Wyoming', 'Guam', 'Connecticut', 'Washington',
       'Colorado', 'Virginia', 'Massachusetts', 'Oregon', 'Maine',
       'Nebraska', 'Illinois', 'Ohio', 'New Mexico', 'Hawaii', 'Texas',
       'District of Columbia', 'New York', 'Iowa', 'Louisiana', 'Vermont',
       'Utah', 'Florida', 'Kentucky', 'New Jersey', 'Oklahoma',
       'South Carolina', 'Idaho', 'Michigan', 'Kansas',
       'IL-City of Chicago', 'IL-Cook County', 'Indiana',
       'IL-Rest of state', 'NY-Rest of state', 'Wisconsin',
       'New Hampshire', 'NY-City of New York', 'Minnesota',
       'PA-Philadelphia', 'Montana', 'TX-Bexar County',
       'TX-Rest of state', 'TX-City of Houston', 'Puerto Rico',
       'PA-Rest of state', 'Rhode Island', 'West Virginia',
       'TX-El Paso County', 'South Dako

In [32]:
# Table 3: Average estimate by age group (Column: Dimension Type) and Survey Year, as average_est_dimtype

average_est_dimtype = vacc_overall.groupby(['Dimension Type', 'Survey Year'], as_index = False).agg(Average_Estimate = ('Estimate (%)', 'mean'))
average_est_dimtype.head()

,Dimension Type,Survey Year,Average_Estimate
0,18-49 Years,2009,75.777778
1,18-49 Years,2010,71.637500
2,18-49 Years,2011,78.116667
3,18-49 Years,2012,77.933333
4,18-49 Years,2013,62.456757


In [33]:
# Pivoted form of average_est_dimtype
average_est_dimtype_pt = average_est_dimtype.pivot_table(index= ['Dimension Type'], columns='Survey Year', values='Average_Estimate')
average_est_dimtype_pt.head()

Survey Year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Dimension Type,,,,,,,,,,,,,,,,
18-49 Years,NaN,75.777778,71.637500,78.116667,77.933333,62.456757,62.680000,64.255556,66.230556,65.300000,68.140000,73.626389,69.500000,67.850000,66.687671,68.800000
18-64 Years,NaN,10.166667,9.875000,16.866667,20.016667,32.071622,35.210000,36.188889,36.508333,37.155556,42.900000,47.352778,47.700000,45.725000,42.391781,47.633333
18-64 Years at Increased Risk,22.960811,24.560811,27.409459,30.085135,29.013514,31.025676,31.479730,34.020270,33.684722,36.754930,29.936105,32.537144,29.012473,28.357725,29.238526,33.172581
50-64 Years,NaN,74.611111,68.287500,73.566667,77.216667,56.389189,59.670000,62.744444,59.750000,63.188889,68.780000,69.959722,73.100000,67.800000,65.465753,70.433333
60-64 Years,NaN,9.560000,10.990000,12.020000,15.271429,NaN,23.042466,28.650000,24.780000,28.126471,27.150000,27.333333,27.252857,32.825000,39.800000,41.143548


In [34]:
# Table 4: Average estimate by race/ethnicity (Column: Dimension) as average_est_dim

vacc_non_overall =  vacc[vacc.Dimension != 'Overall'] # Exclude rows where Dimension = 'Overall'
average_est_dim = vacc_non_overall.groupby(['Dimension', 'Survey Year'], as_index = False).agg(Average_Estimate = ('Estimate (%)', 'mean'))
average_est_dim.head()

,Dimension,Survey Year,Average_Estimate
0,"Black, Non-Hispanic",2008,40.143810
1,"Black, Non-Hispanic",2009,43.022018
2,"Black, Non-Hispanic",2010,44.907965
3,"Black, Non-Hispanic",2011,44.458197
4,"Black, Non-Hispanic",2012,44.931667


In [35]:
# Pivoted form of average_est_dim
average_est_dim_pt = average_est_dim.pivot_table(index= ['Dimension'], columns='Survey Year', values='Average_Estimate')
average_est_dim_pt.head()

Survey Year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Dimension,,,,,,,,,,,,,,,,
"Black, Non-Hispanic",40.143810,43.022018,44.907965,44.458197,44.931667,44.236134,38.172376,47.534815,44.257082,43.333708,45.321488,49.368376,39.139375,44.840566,44.132780,47.230345
Hispanic,39.794521,43.288542,41.739796,43.077064,40.367826,41.401370,36.967153,44.100980,44.677119,42.513548,41.479091,49.245417,36.428261,40.006316,44.511417,42.798630
"Other or Multiple Races, Non-Hispanic",43.299038,46.389381,48.667460,49.577165,47.762500,48.643182,43.663776,51.863380,49.728996,48.676923,49.037324,53.326038,44.800556,48.826400,51.358555,50.216757
"White, Non-Hispanic",46.144218,46.243529,46.199422,49.747561,48.988554,48.143537,46.647303,53.391279,51.741724,52.295197,53.726923,57.866667,50.265888,52.591852,54.507877,54.897872


In [36]:
# Table 5: Year on Year percentage change in Average Estimate by Vaccine, as average_est_yoy

average_est_yoy = vacc_overall.groupby(['Vaccine', 'Survey Year'], as_index = False).agg(Average_Estimate = ('Estimate (%)', 'mean'))
average_est_yoy['YoY % Change'] = average_est_yoy.groupby('Vaccine')['Average_Estimate'].pct_change() * 100

average_est_yoy.head()



,Vaccine,Survey Year,Average_Estimate,YoY % Change
0,Pneumococcal,2008,44.531757,NaN
1,Pneumococcal,2009,45.920270,3.118030
2,Pneumococcal,2010,47.829730,4.158206
3,Pneumococcal,2011,49.635811,3.776064
4,Pneumococcal,2012,48.694595,-1.896244


In [37]:
# Saving each table as separate CSV files in the current working directory

average_est_vacc.to_csv('average_est_vacc.csv', index = False)
est_by_geog.to_csv('est_by_geog.csv', index = False) 
average_est_dimtype.to_csv('average_est_dimtype.csv', index = False)
average_est_dim.to_csv('average_est_dim.csv', index = False)
average_est_yoy.to_csv('average_est_yoy.csv', index = False)